In [ ]:
# Poisson problem on a unit square with Dirichlet boundary conditions
# Manufactured solution.

import numpy as np
import eztfem as ezt
from func import func
from scipy.sparse.linalg import spsolve

In [ ]:
# create mesh
mesh = ezt.quadrilateral2d([20, 20], 'quad9')

In [ ]:
# plot the mesh including curve numbers and point numbers
ezt.plot_points_curves(mesh)

In [ ]:
# define the problem
elementdof = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
                       [2, 2, 2, 2, 2, 2, 2, 2, 2 ]], dtype=int).transpose()
problem = ezt.Problem(mesh, elementdof, nphysq=1)

In [ ]:
# user struct for setting problem coefficients, ...
user = ezt.User()
user.coorsys = 0
user.alpha = 1
user.funcnr = 4
user.func = func

In [ ]:
# define Gauss integration and basis functions
shape = 'quad'
user.xr, user.wg = ezt.gauss_legendre(shape, n=3)
user.phi, user.dphi = ezt.basis_function(shape, 'Q2', user.xr)

In [ ]:
# assemble the system matrix and vector
A, f = ezt.build_system(mesh, problem, ezt.poisson_elem, user)

In [ ]:
# define essential boundary conditions (Dirichlet)
iess = ezt.define_essential(mesh, problem, 'curves', [0, 1, 2, 3])

# fill values for the essential boundary conditions
uess = ezt.fill_system_vector(mesh, problem, 'curves', [0, 1, 2, 3], func,
                                funcnr=3)

# apply essential boundary conditions to the system
ezt.apply_essential(A, f, uess, iess)

In [ ]:
# solve the system
u = spsolve(A.tocsr(), f)

In [ ]:
# compare with exact solution
print('Difference with exact solution:')
uex = ezt.fill_system_vector(mesh, problem, 'nodes',
                                np.arange(mesh.nnodes), func, funcnr=3)

maxdiff = max(abs(u-uex))
print(maxdiff)

In [ ]:
# gradient (dudx,dudy) of the solution
xr = ezt.refcoor_nodal_points(mesh)
[user.phi, user.dphi] = ezt.basis_function('quad', 'Q2', xr)
user.u = u

gradu = ezt.deriv_vector(mesh, problem, ezt.poisson_deriv, user)

In [ ]:
# create a PyVista mesh for visualizing results
mesh_pv = ezt.generate_pyvista_mesh(mesh)

In [ ]:
# plot the mesh in Pyvista
ezt.plot_mesh_pv(mesh_pv, show_edges=True, window_size=(800, 400))

In [ ]:
# plot the solution over a line
points = [[0.0, 0.3, 0.0], [1.0, 0.3, 0.0]]
data_sample = ezt.plot_sol_over_line(mesh_pv, problem, u, points, npoints=200) #, plot_mesh=True)


In [ ]:
# plot the solution field
ezt.plot_sol(mesh_pv, problem, u, show_scalar_bar=True, n_colors=16,
             window_size=(800, 400))